### SVM classifier trained on images (real and generated ones from 4 generators) from ELSA compressed in jpg with quality 40 and coming. The classifier is tested on images from synthbuster

In [1]:
import sys
sys.path.append("../tools")
from utils import load_data_features, load_data_split
from sklearn.svm import LinearSVC
from datasets import load_from_disk
import open_clip
from IPython.display import clear_output
import numpy as np

/home/lsaland/micromamba/envs/clip/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
model, _, preprocess = open_clip.create_model_and_transforms('hf-hub:laion/CLIP-ViT-L-14-DataComp.XL-s13B-b90K',device="cuda")

model.eval()
clear_output()

In [3]:
X_train, y_train = load_data_features("../../data/holistic","train")

In [5]:
def label_conversion(e):
    e["label"] = 1 if e["label"] == "real" else 0
    return e

In [26]:
sb = load_from_disk("../../data/synthbuster_test")
sb = sb.map(label_conversion)
X_test = np.squeeze(np.array(sb["features"]))
y_test = sb["label"]

In [33]:
X_test, y_test = load_data_split("../../data/holistic","test")

In [35]:
clf = LinearSVC(dual="auto")
clf.fit(X_train,y_train).score(X_test,y_test)

0.9508